# Tugas Eksplorasi Pemrograman 3

Pada TEP 3 ini, Anda akan mengeksplorasi *index partitioning* dengan IndexIVFFlat dan *random projection* dengan IndexLSH dari FAISS.
Silakan lakukan eksperimen dan laporkan hasil analisis Anda dengan menambahkan `text `langsung pada notebook yang akan Anda kumpulkan.
Anda dibebaskan untuk mengubah dan menambahkan parameter, melakukan *fine tune* model, atau mencoba teknik *vector indexing* yang lain dari FAISS.

Selamat bereksperimen dan semoga bermanfaat!

Syahrul Apriansyah
2106708311

## Pendahuluan

### Tujuan
Tugas Eksplorasi Pemrograman 3 ini bertujuan untuk memberikan pemahaman mendalam tentang FAISS (Facebook AI Similarity Search), sebuah library yang efisien untuk pencarian kesamaan vektor pada skala besar. Kita akan menjalankan dan menganalisis kode yang telah disediakan dalam Google Colab notebook, dengan fokus pada dua teknik utama: Index Partitioning menggunakan `IndexIVFFlat` dan Random Projection menggunakan `IndexLSH`.


# Reference

Berikut adalah sumber yang dapat Anda pelajari.

- https://www.pinecone.io/learn/series/faiss/faiss-tutorial/
- https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing-random-projection/
- https://www.kaggle.com/code/dailysergey/howtodata-use-faiss-for-similarity-search-cpu

# Setup

In [4]:
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

In [5]:
training_data = pd.read_csv("https://thigm85.github.io/data/cord19/cord19-query-title-label.csv")
training_data

,query,title,label
0,coronavirus origin,Monophyletic Relationship between Severe Acute...,1
1,coronavirus origin,Comprehensive overview of COVID-19 based on cu...,1
2,coronavirus origin,"The SARS, MERS and novel coronavirus (COVID-19...",1
3,coronavirus origin,Evidence for zoonotic origins of Middle East r...,1
4,coronavirus origin,Deadly virus effortlessly hops species,1
...,...,...,...
51387,mRNA vaccine coronavirus,Current Insight into the Novel Coronavirus Dis...,0
51388,mRNA vaccine coronavirus,"Emerging Technologies for Use in the Study, Di...",1
51389,mRNA vaccine coronavirus,Another Set of Coronavirus Vaccine Candidates,1
51390,mRNA vaccine coronavirus,A single-dose live-attenuated YF17D-vectored S...,0


In [6]:
num_query  = 3
num_of_query_with_rels = 3
df = ""
for quer in list(training_data["query"].unique())[:num_query]:
    if len(df)==0:
        df = pd.concat([training_data[training_data["query"]==quer][training_data["label"]==1].head(num_of_query_with_rels), training_data[training_data["query"]==quer][training_data["label"]!=1].head(3)], ignore_index = True)
    else:
        temp_df = pd.concat([training_data[training_data["query"]==quer][training_data["label"]==1].head(num_of_query_with_rels), training_data[training_data["query"]==quer][training_data["label"]!=1].head(3)], ignore_index = True)
        df = pd.concat([df,temp_df], ignore_index = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_26164\971220726.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = pd.concat([training_data[training_data["query"]==quer][training_data["label"]==1].head(num_of_query_with_rels), training_data[training_data["query"]==quer][training_data["label"]!=1].head(3)], ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_26164\971220726.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = pd.concat([training_data[training_data["query"]==quer][training_data["label"]==1].head(num_of_query_with_rels), training_data[training_data["query"]==quer][training_data["label"]!=1].head(3)], ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_26164\971220726.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_df = pd.concat([training_data[training_data["query"]==quer][training_data["label"]==1].head(num_of_query_with_rels), training_da

Pada bagian kode di atas data diproses untuk mengumpulkan judul artikel yang relevan dan tidak relevan berdasarkan query. Pendekatan ini menciptakan keseimbangan antara hasil positif dan negatif, penting untuk evaluasi model pencarian. Dengan cara ini, dapat dilakukan analisis atau pelatihan model yang lebih akurat, khususnya dalam konteks sistem rekomendasi atau pencarian berbasis teks.

In [7]:
df = df.sample(frac=1)
df.shape

(18, 3)

In [8]:
sentences = list(df["title"].unique())
indonesia_doc = ["Pentingnya olahraga untuk kesehatan mental juga tidak boleh diabaikan. Aktivitas fisik dapat mengurangi tingkat stres dan kecemasan dengan melepaskan endorfin, hormon bahagia, yang dapat meningkatkan suasana hati. Selain itu, olahraga teratur juga dapat meningkatkan kognisi dan fungsi otak, membantu mencegah penurunan kognitif terkait usia. Dengan manfaat ini, olahraga menjadi elemen integral dalam menjaga keseimbangan holistik antara kesehatan fisik dan mental.",
                 "Selain penerapan kecerdasan buatan, teknologi terbaru juga mencakup pengembangan terapi gen, yang memiliki potensi untuk menyembuhkan penyakit genetik dan kanker. Terapi gen menggunakan manipulasi genetik untuk mengubah atau memperbaiki sel-sel tubuh manusia. Di samping itu, penggunaan robotika dalam pembedahan semakin meningkat, memungkinkan presisi yang lebih tinggi dan pemulihan yang lebih cepat. Dengan terus berkembangnya teknologi di bidang kesehatan, harapannya adalah peningkatan efektivitas pengobatan dan peningkatan kualitas hidup pasien.",
                 "Perubahan iklim juga memberikan dampak signifikan pada keanekaragaman hayati. Pergeseran suhu dan pola hujan mempengaruhi habitat alami, menyebabkan migrasi spesies dan bahkan kepunahan. Selain itu, terjadi acidifikasi laut akibat peningkatan kadar karbon dioksida, yang berdampak buruk pada organisme laut dengan cangkang kalsium, seperti karang dan moluska. Dengan menyadari konsekuensi ini, penting untuk mempromosikan praktik berkelanjutan dan upaya konservasi untuk melindungi keanekaragaman hayati dan memitigasi dampak perubahan iklim di seluruh dunia.",
                 "Tidak hanya bermanfaat untuk kesehatan fisik dan mental, olahraga juga dapat memperkuat hubungan sosial. Melalui partisipasi dalam kegiatan olahraga bersama, individu dapat membangun jejaring sosial, memperluas lingkaran pertemanan, dan meningkatkan keterlibatan dalam komunitas. Terlibat dalam tim olahraga atau kegiatan kelompok lainnya tidak hanya meningkatkan rasa solidaritas tetapi juga memberikan kesempatan untuk belajar nilai-nilai seperti kerjasama, kepemimpinan, dan toleransi, yang dapat membentuk karakter positif.",
                 "Teknologi kesehatan terbaru tidak hanya berkaitan dengan pengembangan perangkat keras, tetapi juga mengenai perangkat lunak dan aplikasi yang mendukung manajemen kesehatan pribadi. Aplikasi kesehatan digital dan platform telemedicine semakin memudahkan pengguna untuk memantau kondisi kesehatan mereka, menjadikan pencegahan sebagai fokus utama. Selain itu, penggunaan teknologi blockchain mulai diintegrasikan dalam sistem rekam medis elektronik, meningkatkan keamanan dan aksesibilitas data pasien dengan memberikan kontrol lebih besar kepada individu terkait informasi kesehatan mereka.",
                 "Dampak perubahan iklim tidak hanya terbatas pada ekosistem darat dan laut, tetapi juga berpengaruh signifikan pada sektor ekonomi dan keberlanjutan. Pertanian, misalnya, menghadapi tantangan seperti penurunan hasil pertanian karena perubahan pola hujan yang tidak stabil. Selain itu, sektor pariwisata juga terdampak oleh kerusakan lingkungan, seperti bleaching karang akibat suhu laut yang meningkat. Upaya untuk mengatasi perubahan iklim tidak hanya penting untuk melindungi lingkungan alam, tetapi juga untuk menjaga ketahanan ekonomi dan kehidupan manusia di berbagai sektor.",
                 "Olahraga memiliki manfaat besar bagi kesehatan tubuh manusia. Seiring gaya hidup modern yang cenderung kurang aktif, melakukan olahraga secara teratur dapat membantu menjaga berat badan yang sehat dan mengurangi risiko obesitas. Selain itu, olahraga juga dapat meningkatkan kesehatan jantung dengan memperkuat otot-otot jantung dan meningkatkan peredaran darah. Manfaat lainnya termasuk peningkatan stamina, peningkatan kualitas tidur, dan peningkatan mood melalui pelepasan endorfin selama aktivitas fisik.",
                 "Inovasi teknologi terbaru telah mengubah landscape bidang kesehatan secara signifikan. Salah satu perkembangan penting adalah penerapan kecerdasan buatan (AI) dalam diagnosis medis. Teknologi ini memungkinkan analisis data yang cepat dan akurat untuk membantu dokter dalam mendiagnosis penyakit. Selain itu, telemedicine juga semakin populer, memungkinkan konsultasi medis jarak jauh melalui platform digital. Pengembangan teknologi wearable seperti smartwatch juga memungkinkan pengguna untuk memantau kondisi kesehatan mereka secara real-time.",
                 "Perubahan iklim memiliki dampak serius terhadap lingkungan dan kehidupan manusia. Peningkatan suhu global telah menyebabkan perubahan ekosistem dan distribusi spesies. Kenaikan permukaan air laut akibat schmelting es polar mengancam puluhan ribu pulau dan kota pesisir. Perubahan pola cuaca ekstrem, seperti banjir dan kekeringan, juga semakin sering terjadi, mengancam keamanan pangan dan keberlanjutan pertanian. Dengan memahami dampak ini, upaya global untuk mengurangi emisi gas rumah kaca menjadi krusial untuk menjaga kelestarian bumi bagi generasi mendatang."]
sentences.extend(indonesia_doc)

In [9]:
indonesia_queries = [
    "Manfaat olahraga bagi kesehatan",
    "Teknologi terbaru di bidang kesehatan",
    "Pengaruh perubahan iklim terhadap lingkungan"
]

In [10]:
model = SentenceTransformer('LaBSE')
sentence_embeddings = model.encode(sentences)
print(sentence_embeddings)

[[ 0.02416993 -0.01509913  0.01173402 ... -0.00607438 -0.00010206
   0.00514793]
 [ 0.02498717 -0.06530681 -0.00934507 ... -0.0139977  -0.0591126
   0.01069342]
 [-0.02549892 -0.04476077 -0.0045252  ...  0.02719675 -0.03129315
   0.01236853]
 ...
 [-0.05606006  0.04707864 -0.03916391 ...  0.03355038 -0.05798845
  -0.06005105]
 [ 0.04067453 -0.01359548  0.04488348 ...  0.05443221 -0.03191138
  -0.05487609]
 [-0.03776889 -0.0442142  -0.05109699 ...  0.02286782 -0.06238412
  -0.06133436]]


In [11]:
sentence_embeddings.shape

(27, 768)

In [12]:
d = sentence_embeddings.shape[1]

In [13]:
query1 = str(df["query"].iloc[0])
print(query1)

coronavirus response to weather changes


In [14]:
xq = model.encode(query1)
xq = xq[np.newaxis, :]
xq.shape

(1, 768)

Kode di atas bertujuan untuk melakukan proses embedding pada teks menggunakan model 'LaBSE' dari SentenceTransformer. Berikut adalah analisis  dari proses dan tujuan kode tersebut:

### Tujuan Utama
- Tujuan utama dari kode ini adalah untuk mengubah teks menjadi vektor embedding yang kemudian dapat digunakan dalam algoritma pencarian vektor seperti FAISS. Embedding ini membawa informasi semantik dari teks, yang sangat penting untuk pencarian dan analisis yang berbasis pada makna teks, bukan hanya kata-kata tertentu.

### Proses Embedding
- Kode pertama mengumpulkan judul-judul unik dari sebuah dataset (`df`) dan menambahkannya dengan dokumen tambahan dalam bahasa Indonesia. Ini menciptakan dataset teks yang kaya dan multibahasa.
- `SentenceTransformer('LaBSE')` digunakan untuk mengubah teks ini menjadi embeddings. 'LaBSE' adalah model yang efisien untuk pemahaman bahasa lintas bahasa, memungkinkan embedding yang dibuat mampu menangkap makna dalam berbagai bahasa.
- Setiap teks diubah menjadi vektor 768 dimensi, memberikan representasi numerik yang kaya dari teks asli.

### Multibahasa dan Multitopik
- Integrasi teks dari topik yang beragam (seperti kesehatan, olahraga, perubahan iklim) dan dalam bahasa yang berbeda (bahasa Inggris dan Indonesia) menunjukkan upaya untuk membuat model yang mampu menangani beragam jenis query dalam berbagai bahasa. Ini penting untuk aplikasi pencarian dalam skenario dunia nyata yang sering menghadapi variasi bahasa dan topik.


In [15]:
def encoding_query(query):
    xquery = model.encode(query)
    xquery = xquery[np.newaxis, :]
    return xquery

In [16]:
def display_query_result(Isearch):
    counter = 0
    for i in Isearch[0]:
        counter+=1
        res = sentences[i]
        print(counter,".",res)

# Partition Index with IndexIVFFlat

In [17]:
# Set the number of cells
nlist = 8
# Partitions the data into cells, where each cell is associated with a separate index structure
quantizer = faiss.IndexFlatL2(d)

In [18]:
# Using this method, we will take a query vector, identify the Voronoi cells it belongs to, and then use IndexFlatL2 (or another metric)
# to search between the query vector and all other vectors that fall into that specific cell, so we reduced the scope of our search.
indexIVFFlat = faiss.IndexIVFFlat(quantizer, d, nlist)
indexIVFFlat.train(sentence_embeddings)
indexIVFFlat.add(sentence_embeddings)

In [19]:
# Number of embeddings indexed
indexIVFFlat.ntotal

27

In [20]:
# Set the number of nearby cells to search
indexIVFFlat.nprobe = 10

In [21]:
%%time
# Outputs the index positions I of our k best matches
# and the distances D between those best matches and our query vector
D, I = indexIVFFlat.search(xq, k=10)
print(f"Index positions I: {I}")
print(f"Distances D: {D}")

Index positions I: [[10 13  4 12 14  6  2 20 26 15]]
Distances D: [[0.7985459 1.063704  1.0947502 1.1140382 1.1389596 1.1574091 1.1719561
  1.1956993 1.2213402 1.2458675]]
CPU times: total: 15.6 ms
Wall time: 2 ms


In [22]:
[f'{i}: {sentences[i]}' for i in I[0]]

['10: Association between climate variables and global transmission oF SARS-CoV-2',
 '13: Recommendations for coronavirus infection in rheumatic diseases treated with biologic therapy',
 '4: Cross-immunity between respiratory coronaviruses may limit COVID-19 fatalities',
 '12: An outbreak of COVID‐19 caused by a new coronavirus: what we know so far',
 '14: Temperature Decreases Spread Parameters of the New Covid-19 Case Dynamics',
 '6: Comparative analysis of innate immune responses following infection of newborn calves with bovine rotavirus and bovine coronavirus.',
 '2: Effect of Environmental Conditions on SARS-CoV-2 Stability in Human Nasal Mucus and Sputum',
 '20: Perubahan iklim juga memberikan dampak signifikan pada keanekaragaman hayati. Pergeseran suhu dan pola hujan mempengaruhi habitat alami, menyebabkan migrasi spesies dan bahkan kepunahan. Selain itu, terjadi acidifikasi laut akibat peningkatan kadar karbon dioksida, yang berdampak buruk pada organisme laut dengan cangkang

In [23]:
df[df["query"]==query1][df["label"]==1]

C:\Users\USER\AppData\Local\Temp\ipykernel_26164\2185750697.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df["query"]==query1][df["label"]==1]


,query,title,label
8,coronavirus response to weather changes,Effect of Environmental Conditions on SARS-CoV...,1
6,coronavirus response to weather changes,Association between climate variables and glob...,1
7,coronavirus response to weather changes,Temperature Decreases Spread Parameters of the...,1


Kode yang di atas adalah implementasi dari sistem pencarian vektor menggunakan FAISS, khususnya menggunakan metode IndexIVFFlat.Berikut adalah analisis dari proses dan tujuan kode tersebut:

### Strategi dan Implementasi FAISS IndexIVFFlat

1. **Pengaturan IndexIVFFlat**: 
    - Dengan menggunakan IndexIVFFlat, yang merupakan metode FAISS untuk partisi data vektor, sistem menciptakan struktur yang memungkinkan pencarian yang lebih cepat dan efisien.
    - `nlist = 8` menentukan jumlah cell atau region yang index akan buat. Ini mempengaruhi cara data disimpan dan dicari dalam index.

2. **Pembuatan Quantizer**:
    - `quantizer = faiss.IndexFlatL2(d)` digunakan untuk menginisialisasi quantizer yang diperlukan oleh IndexIVFFlat. Quantizer ini bertindak sebagai struktur dasar untuk mempartisi ruang vektor.

3. **Training dan Menambahkan Data**:
    - `indexIVFFlat.train(sentence_embeddings)` dan `indexIVFFlat.add(sentence_embeddings)` adalah langkah-langkah penting untuk melatih index dengan embeddings yang dihasilkan dan kemudian menambahkan embeddings tersebut ke dalam index.

4. **Penetapan nprobe**:
    - `indexIVFFlat.nprobe = 10` menetapkan jumlah cell terdekat yang akan dicari dalam proses pencarian. Ini adalah faktor kunci yang mempengaruhi keseimbangan antara kecepatan pencarian dan akurasi.

### Evaluasi dan Analisis Hasil

1. **Proses Pencarian**:
    - Melalui `D, I = indexIVFFlat.search(xq, k=10)`, sistem mencari 10 vektor terdekat dengan vektor query. Hasilnya adalah indeks dan jarak dari vektor terdekat.

2. **Analisis Hasil Pencarian**:
    - Output menunjukkan indeks dan jarak vektor terdekat. Indeks ini merujuk pada posisi embeddings dalam dataset yang diberikan, dan jarak menunjukkan seberapa dekat vektor query dengan vektor di index.

3. **Konteks dalam Pencarian**:
    - Melalui `df[df["query"]==query1][df["label"]==1]`, bisa dilihat judul artikel yang dianggap relevan dengan query yang diberikan. Ini memberikan konteks untuk memahami seberapa efektif sistem dalam menemukan konten yang relevan.


In [24]:
def search_with_indexIVFFlat(query,number_of_result=5):
    x_query = encoding_query(query)
    nlist = 8
    quantizer = faiss.IndexFlatL2(d)

    indexIVFFlat = faiss.IndexIVFFlat(quantizer, d, nlist)
    indexIVFFlat.train(sentence_embeddings)
    indexIVFFlat.add(sentence_embeddings)

    indexIVFFlat.nprobe = 10

    Dsearch, Isearch = indexIVFFlat.search(x_query, k=number_of_result)
    display_query_result(Isearch)


### Integrasi FAISS dalam Pencarian Semantik
- Fungsi `search_with_indexIVFFlat` menggambarkan penerapan FAISS untuk mencari vektor yang paling cocok dengan query yang diberikan. Proses ini dimulai dengan meng-encode query menjadi vektor menggunakan model SentenceTransformer yang sama yang digunakan untuk meng-encode judul artikel.

### Konfigurasi FAISS IndexIVFFlat
- Dalam konfigurasi IndexIVFFlat, `nlist` diatur menjadi 8, yang mendefinisikan bagaimana data dibagi menjadi cell atau cluster. Ini penting untuk mengoptimalkan pencarian dengan membatasi ruang pencarian.
- `nprobe` diatur ke 10, yang menentukan berapa banyak cell terdekat yang akan dipertimbangkan dalam pencarian. Ini menyeimbangkan antara kecepatan pencarian dan kemungkinan menemukan hasil yang paling relevan.


## Free text Query IndexIVFFlat
Anda dapat mencoba query free text pada pencarian menggunakan teknik index IndexIVFFlat dengan mengganti teks pada parameter dibawah

In [25]:
search_with_indexIVFFlat("fungsi olahraga bagi kesehatan")

1 . Olahraga memiliki manfaat besar bagi kesehatan tubuh manusia. Seiring gaya hidup modern yang cenderung kurang aktif, melakukan olahraga secara teratur dapat membantu menjaga berat badan yang sehat dan mengurangi risiko obesitas. Selain itu, olahraga juga dapat meningkatkan kesehatan jantung dengan memperkuat otot-otot jantung dan meningkatkan peredaran darah. Manfaat lainnya termasuk peningkatan stamina, peningkatan kualitas tidur, dan peningkatan mood melalui pelepasan endorfin selama aktivitas fisik.
2 . Pentingnya olahraga untuk kesehatan mental juga tidak boleh diabaikan. Aktivitas fisik dapat mengurangi tingkat stres dan kecemasan dengan melepaskan endorfin, hormon bahagia, yang dapat meningkatkan suasana hati. Selain itu, olahraga teratur juga dapat meningkatkan kognisi dan fungsi otak, membantu mencegah penurunan kognitif terkait usia. Dengan manfaat ini, olahraga menjadi elemen integral dalam menjaga keseimbangan holistik antara kesehatan fisik dan mental.
3 . Tidak hanya b

In [26]:
search_with_indexIVFFlat("Pengaruh perubahan iklim terhadap lingkungan")

1 . Perubahan iklim memiliki dampak serius terhadap lingkungan dan kehidupan manusia. Peningkatan suhu global telah menyebabkan perubahan ekosistem dan distribusi spesies. Kenaikan permukaan air laut akibat schmelting es polar mengancam puluhan ribu pulau dan kota pesisir. Perubahan pola cuaca ekstrem, seperti banjir dan kekeringan, juga semakin sering terjadi, mengancam keamanan pangan dan keberlanjutan pertanian. Dengan memahami dampak ini, upaya global untuk mengurangi emisi gas rumah kaca menjadi krusial untuk menjaga kelestarian bumi bagi generasi mendatang.
2 . Perubahan iklim juga memberikan dampak signifikan pada keanekaragaman hayati. Pergeseran suhu dan pola hujan mempengaruhi habitat alami, menyebabkan migrasi spesies dan bahkan kepunahan. Selain itu, terjadi acidifikasi laut akibat peningkatan kadar karbon dioksida, yang berdampak buruk pada organisme laut dengan cangkang kalsium, seperti karang dan moluska. Dengan menyadari konsekuensi ini, penting untuk mempromosikan prak

- Dua contoh pencarian, satu terkait "olahraga" dan yang lainnya tentang "perubahan iklim", menunjukkan bagaimana sistem mencari artikel yang relevan dari dataset.
- Hasil yang dihasilkan menunjukkan keberhasilan sistem dalam mengidentifikasi artikel yang relevan dari dataset berdasarkan query yang diberikan. Ini menunjukkan efektivitas embeddings dan algoritma pencarian dalam mengenali dan menyajikan konten yang relevan.
- Implementasi ini menunjukkan kemajuan signifikan dalam membangun sistem pencarian teks yang cerdas dan adaptif. Dengan mengintegrasikan teknologi embeddings semantik dan FAISS, sistem ini mampu memproses dan merespons query pencarian dengan cara yang lebih intuitif dan kontekstual.
- Kinerja sistem dalam menangani query dalam bahasa yang berbeda juga menunjukkan kemampuannya dalam lingkungan multibahasa, menambahkan lapisan kegunaan yang penting untuk aplikasi global.

# Random Projection for LSH

In [27]:
# Set the number of hyperplanes and binary vals to produce
nbits = 128

In [28]:
# At its core, it is a hashing function that allows us to group similar items into the same hash buckets.
# Using the random projection method, we will reduce our highly-dimensional vectors into low-dimensionality binary vectors.
# Once we have these binary vectors, we can measure the distance between them.
indexLSH = faiss.IndexLSH(d, nbits)
indexLSH.add(sentence_embeddings)

- **Konfigurasi LSH**: `nbits = 128` menentukan jumlah hyperplanes yang digunakan untuk proyeksi acak. Ini adalah parameter kunci yang menentukan bagaimana data vektor diubah menjadi representasi biner.
- **Pembuatan Index LSH**: `indexLSH = faiss.IndexLSH(d, nbits)` menginisialisasi index LSH dengan dimensi vektor dan jumlah hyperplanes yang ditentukan. Index ini bertugas mengubah vektor embedding ke dalam representasi biner melalui proyeksi acak.


In [29]:
%%time
# Outputs the index positions I of our k best matches
# and the distances D between those best matches and our query vector
D, I = indexLSH.search(xq, k=10)
print(f"Index positions I: {I}")
print(f"Distances D: {D}")

Index positions I: [[14 10 12 13  2  9 11 15  7 16]]
Distances D: [[30. 38. 40. 43. 44. 44. 44. 44. 46. 47.]]
CPU times: total: 15.6 ms
Wall time: 2 ms


In [30]:
[f'{i}: {sentences[i]}' for i in I[0]]

['14: Temperature Decreases Spread Parameters of the New Covid-19 Case Dynamics',
 '10: Association between climate variables and global transmission oF SARS-CoV-2',
 '12: An outbreak of COVID‐19 caused by a new coronavirus: what we know so far',
 '13: Recommendations for coronavirus infection in rheumatic diseases treated with biologic therapy',
 '2: Effect of Environmental Conditions on SARS-CoV-2 Stability in Human Nasal Mucus and Sputum',
 '9: The SARS, MERS and novel coronavirus (COVID-19) epidemics, the newest and biggest global health threats: what lessons have we learned?',
 '11: P136 Combined action of type I and type III IFN restricts initial replication of SARS-coronavirus in the lung but fails to inhibit systemic virus spread',
 '15: Prevention and treatment of COVID-19 disease by controlled modulation of innate immunity',
 '7: Comprehensive overview of COVID-19 based on current evidence',
 '16: Boceprevir, GC-376, and calpain inhibitors II, XII inhibit SARS-CoV-2 viral rep

- **Menambahkan Embeddings**: `indexLSH.add(sentence_embeddings)` menambahkan embeddings teks ke index LSH. Ini mempersiapkan data untuk pencarian tetangga terdekat.
- **Pencarian Tetangga Terdekat**: `D, I = indexLSH.search(xq, k=10)` melakukan pencarian untuk menemukan 10 tetangga terdekat dari query yang diberikan. Hasilnya adalah indeks dan jarak dari vektor terdekat.
- Indeks mengacu pada posisi embeddings dalam dataset, sementara jarak (dalam kasus ini, jarak Hamming karena menggunakan LSH) menunjukkan seberapa mirip query dengan vektor di dataset. Dengan demikian, hasil pencarian memberikan informasi tentang artikel mana yang paling relevan dengan query berdasarkan representasi biner mereka.


In [31]:
df[df["query"]==query1][df["label"]==1]

C:\Users\USER\AppData\Local\Temp\ipykernel_26164\2185750697.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df["query"]==query1][df["label"]==1]


,query,title,label
8,coronavirus response to weather changes,Effect of Environmental Conditions on SARS-CoV...,1
6,coronavirus response to weather changes,Association between climate variables and glob...,1
7,coronavirus response to weather changes,Temperature Decreases Spread Parameters of the...,1


Membandingkan hasil pencarian dengan artikel yang dianggap relevan (`df[df["query"]==query1][df["label"]==1]`) memberikan wawasan tentang akurasi sistem dalam menemukan konten yang relevan. Terlihat bahwa artikel yang dianggap relevan dengan query ditemukan dalam hasil pencarian, menunjukkan keberhasilan sistem dalam menemukan konten yang relevan. Ini menunjukkan efektivitas embeddings dan algoritma pencarian dalam mengenali dan menyajikan konten yang relevan. 

In [32]:
def search_with_LSH(query,number_of_result=5):
    x_query = encoding_query(query)
    nbits = 128
    indexLSH = faiss.IndexLSH(d, nbits)
    indexLSH.add(sentence_embeddings)
    Dsearch, Isearch = indexLSH.search(x_query, k=number_of_result)
    display_query_result(Isearch)

- Fungsi `search_with_LSH` dirancang untuk mengubah query teks menjadi vektor embedding dan menggunakan LSH untuk menemukan cocokan yang paling relevan dalam dataset.
- Dalam LSH, `nbits = 128` mendefinisikan jumlah bit yang digunakan dalam representasi hash, yang mempengaruhi bagaimana data dipartisi dan disimpan. Ini adalah langkah penting untuk mengurangi dimensi data tanpa mengorbankan terlalu banyak informasi penting.
- `indexLSH = faiss.IndexLSH(d, nbits)` menginisialisasi index LSH dengan dimensi vektor dan jumlah bit yang ditentukan. Index ini bertugas mengubah vektor embedding ke dalam representasi biner melalui proyeksi acak.
- `indexLSH.add(sentence_embeddings)` menambahkan embeddings teks ke index LSH. Ini mempersiapkan data untuk pencarian tetangga terdekat.
- Setelah embeddings teks ditambahkan ke index LSH, fungsi tersebut mencari cocokan terdekat untuk query yang diberikan. Ini dilakukan dengan mencari di hash bucket yang sesuai, yang secara signifikan mengurangi ruang pencarian dibandingkan dengan pencarian di seluruh dataset.
- Hasil yang ditampilkan menunjukkan top 5 hasil pencarian berdasarkan query

## Free text Query LSH
Anda dapat mencoba query free text pada pencarian menggunakan teknik index LSH dengan mengganti teks pada parameter dibawah

In [33]:
search_with_LSH("olahraga itu penting")

1 . Tidak hanya bermanfaat untuk kesehatan fisik dan mental, olahraga juga dapat memperkuat hubungan sosial. Melalui partisipasi dalam kegiatan olahraga bersama, individu dapat membangun jejaring sosial, memperluas lingkaran pertemanan, dan meningkatkan keterlibatan dalam komunitas. Terlibat dalam tim olahraga atau kegiatan kelompok lainnya tidak hanya meningkatkan rasa solidaritas tetapi juga memberikan kesempatan untuk belajar nilai-nilai seperti kerjasama, kepemimpinan, dan toleransi, yang dapat membentuk karakter positif.
2 . Association between climate variables and global transmission oF SARS-CoV-2
3 . Pentingnya olahraga untuk kesehatan mental juga tidak boleh diabaikan. Aktivitas fisik dapat mengurangi tingkat stres dan kecemasan dengan melepaskan endorfin, hormon bahagia, yang dapat meningkatkan suasana hati. Selain itu, olahraga teratur juga dapat meningkatkan kognisi dan fungsi otak, membantu mencegah penurunan kognitif terkait usia. Dengan manfaat ini, olahraga menjadi elem

- Hasil pencarian menunjukkan variasi topik yang cukup lebar, dari manfaat olahraga bagi kesehatan dan hubungan sosial hingga topik yang tidak langsung terkait seperti perubahan iklim. Ini mungkin menunjukkan bahwa sementara LSH efektif dalam mengurangi waktu pencarian, ia mungkin kurang presisi dalam hal kesesuaian semantik dibandingkan dengan metode pencarian lainnya.
- Variasi dalam hasil pencarian juga bisa mencerminkan kompleksitas dan multivariasi dari embeddings yang digunakan. Karena embeddings semantik menangkap berbagai aspek makna, sistem mungkin menemukan koneksi yang tidak langsung antara query dan artikel dalam dataset.
- Penggunaan teknik ini sangat berguna dalam kasus di mana respons cepat lebih diutamakan daripada presisi mutlak, seperti dalam sistem rekomendasi awal atau pencarian eksploratif dalam dataset besar.